In [1]:
!apt-get update # Update apt-get repository.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # Install Java.
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # Download Apache Sparks.
!tar xf spark-3.1.1-bin-hadoop3.2.tgz # Unzip the tgz file.
!pip install -q findspark # Install findspark. Adds PySpark to the System path during runtime.

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

!ls

# Initialize findspark
import findspark
findspark.init()

# Create a PySpark session
from pyspark import SparkContext, SparkConf
import collections

conf=SparkConf().setMaster("local").setAppName("RatingHistogram")
sc = SparkContext(conf=conf)

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,589 kB]
Get:14 https://r2u.stat.illinois.edu/ubunt

In [2]:
lines=sc.textFile("/content/u.data")

In [3]:
for i in lines.collect():
  print(i)

Streaming output truncated to the last 5000 lines.
589	326	1	883352600
561	160	3	885808904
757	426	3	888467270
943	230	1	888693158
616	327	3	891224558
887	756	5	881379094
768	301	5	883835026
880	186	4	880174808
851	1254	1	875730895
178	268	4	884837324
795	154	3	881529904
276	413	3	877934705
551	660	3	892783672
758	13	5	881977205
67	1047	3	875379750
495	1	4	888632943
918	340	1	891986174
556	321	4	882135994
521	174	4	884478721
866	303	4	891221165
416	122	3	886315885
5	373	3	875635907
918	971	4	891987780
534	118	4	877807935
875	428	4	876465112
620	393	5	889988196
483	258	4	878950353
802	569	3	875985840
256	597	4	882152509
49	404	3	888067765
500	13	5	883865232
648	1337	3	884367366
454	387	2	888267279
221	1208	3	875247565
498	77	2	881961627
924	429	4	886760020
815	229	3	878695527
650	174	4	891369479
525	147	3	881085893
848	199	5	887042341
755	301	3	882569771
697	237	5	882622414
559	300	4	891035137
775	333	4	891033022
798	815	5	875295695
398	8	3	875716709
456	324	4	881372687
727	472	2	883709

In [4]:
ratings=lines.map(lambda r: r.split()[2])

In [5]:
result=ratings.countByValue()

In [6]:
sorted_result=collections.OrderedDict(sorted(result.items()))
for k, v in sorted_result.items():
  print(k, v)

1 6110
2 11370
3 27145
4 34174
5 21201


**FIND OUT THE AVERAGE FRIENDS BY EXAMPLE**

In [7]:
rdd=sc.textFile("/content/fakefriends.csv")
for i in rdd.collect():
  print(i)

0,Will,33,385
1,Jean-Luc,26,2
2,Hugh,55,221
3,Deanna,40,465
4,Quark,68,21
5,Weyoun,59,318
6,Gowron,37,220
7,Will,54,307
8,Jadzia,38,380
9,Hugh,27,181
10,Odo,53,191
11,Ben,57,372
12,Keiko,54,253
13,Jean-Luc,56,444
14,Hugh,43,49
15,Rom,36,49
16,Weyoun,22,323
17,Odo,35,13
18,Jean-Luc,45,455
19,Geordi,60,246
20,Odo,67,220
21,Miles,19,268
22,Quark,30,72
23,Keiko,51,271
24,Julian,25,1
25,Ben,21,445
26,Julian,22,100
27,Leeta,42,363
28,Martok,49,476
29,Nog,48,364
30,Keiko,50,175
31,Miles,39,161
32,Nog,26,281
33,Dukat,53,197
34,Jean-Luc,43,249
35,Beverly,27,305
36,Kasidy,32,81
37,Geordi,58,21
38,Deanna,64,65
39,Morn,31,192
40,Odo,52,413
41,Hugh,67,167
42,Brunt,54,75
43,Guinan,58,345
44,Nerys,35,244
45,Dukat,52,77
46,Morn,25,96
47,Brunt,24,49
48,Nog,20,1
49,Ezri,40,254
50,Quark,51,283
51,Lwaxana,36,212
52,Beverly,19,269
53,Geordi,62,31
54,Brunt,19,5
55,Keiko,41,278
56,Gowron,44,194
57,Odo,57,294
58,Hugh,59,158
59,Morn,59,284
60,Geordi,20,100
61,Kasidy,62,442
62,Keiko,69,9
63,Jean-Luc,58,54
64,El

In [9]:
def parseInt(rdd):
  fields=rdd.split(",")
  age=int(fields[2])
  numFriends=int(fields[3])
  return (age, numFriends)

rdd1=rdd.map(parseInt)
rdd2=rdd1.mapValues(lambda x:(x,1)).reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1]))

In [11]:
#for i in rdd2.collect():
  #print(i)

#for average
rdd3=rdd2.mapValues(lambda x:x[0]/x[1])

**MINIMUM TEMPARATURE BY LOCATION**

In [17]:
rdd = sc.textFile("/content/1800.csv")

def parseLine(line):
    fields = line.split(',')
    stationID = fields[0]
    entryType = fields[2]
    temperature = float(fields[3]) * 0.1 * (9.0/5.0) + 32.0
    return (stationID, entryType, temperature)

rdd1 = rdd.map(parseLine)
rdd2 = rdd1.filter(lambda x: 'TMIN' in x[1])
rdd3 = rdd2.map(lambda y: (y[0], y[2]))  # Return a tuple
rdd4=rdd3.reduceByKey(lambda x,y: min(x,y))

for i in rdd4.collect():
  print(i)


('ITE00100554', 5.359999999999999)
('EZE00100082', 7.699999999999999)


**COUNT THE WORDS IN A BOOK**

In [23]:
import re

rdd = sc.textFile("/content/Book")

def normalize_words(line):
    return re.compile(r'\W+', re.UNICODE).split(line.lower())

rdd1 = rdd.flatMap(normalize_words)
words = rdd1.countByValue()

for word, count in words.items():
    print(word, count)


self 111
employment 75
building 33
an 178
internet 26
business 383
of 970
one 100
achieving 1
financial 17
and 934
personal 48
freedom 41
through 57
a 1191
lifestyle 44
technology 11
by 122
frank 11
kane 10
 772
copyright 3
2015 4
all 137
rights 3
reserved 2
worldwide 4
contents 1
disclaimer 2
preface 2
part 33
i 387
making 25
the 1292
big 42
decision 12
overcoming 2
inertia 2
fear 3
failure 3
career 31
indoctrination 5
carrot 4
on 428
stick 6
ego 3
protection 7
your 1420
employer 44
as 343
security 8
blanket 2
why 25
it 649
s 391
worth 39
unlimited 6
growth 39
potential 38
investing 16
in 616
yourself 78
not 203
someone 62
else 33
no 76
dependencies 6
commute 14
to 1828
live 25
where 53
you 1878
want 122
work 144
when 102
how 163
is 560
for 537
flowchart 4
should 69
even 104
consider 26
having 30
safety 7
net 13
planning 16
health 35
care 24
assessment 4
quiz 4
ii 2
happen 13
designing 4
fallacy 2
introducing 3
ideal 3
case 26
study 4
sundog 24
software 60
other 78
ideas 27
key 6
poin

**SORT THE COUNT KEYS**

In [30]:
import re

rdd = sc.textFile("/content/Book")  # Ensure the path is correct

def normalize_words(line):
    return re.compile(r'\W+', re.UNICODE).split(line.lower())

rdd1 = rdd.flatMap(normalize_words)
words = rdd1.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)

# Sort by the word (first element of the tuple)
sorted_words = words.map(lambda x: (x[1], x[0])).sortByKey()

result = sorted_words.collect()

for count, word in result:
    print(word, count)


achieving 1
contents 1
preparation 1
skillset 1
determination 1
confidence 1
strike 1
blame 1
devoted 1
commuted 1
complaint 1
rewarded 1
role 1
marriage 1
combat 1
secondary 1
ultimatum 1
weeks 1
walked 1
matches 1
nor 1
requirement 1
43 1
broke 1
gloat 1
heart 1
attack 1
65 1
retire 1
smarts 1
tenacity 1
discarding 1
bold 1
psyche 1
rebel 1
magnitude 1
justify 1
surprising 1
fell 1
laid 1
downsized 1
imagined 1
pursuing 1
roof 1
starving 1
foreclosed 1
returning 1
prototyped 1
proven 1
quits 1
sba 1
tenure 1
secure 1
28 1
moderately 1
americans 1
representing 1
14 1
fringe 1
internal 1
religious 1
cults 1
brainwashing 1
beliefs 1
questioning 1
instill 1
youth 1
grew 1
teachers 1
grades 1
graduated 1
absorbed 1
culture 1
promotes 1
indoctrinated 1
landed 1
children 1
terrifying 1
barely 1
mouths 1
fulfill 1
thrust 1
foam 1
cup 1
ramen 1
noodles 1
desire 1
alternate 1
paths 1
commensurate 1
struggling 1
spin 1
banking 1
impacts 1
materially 1
rationally 1
tiny 1
proceeds 1
answered 1
a